## This notebook takes the QC'd fasta file with all seqs and clusters it to OTUs

In [13]:
# Pulling out the sample identifier.
# It is adding a portion to the finalQC file that has the barcode label.
# Then we can use this later
# Now we will see how these reads map to the defined centroids (after removing EuK, etc.)
# Basically, we cut, cut, refined our fasta to make our OTU centroids.
# THEN, we went back to our original QC'd total fasta file and will throw it all against these nicely defined seeds.
# Anything that doesn't match, we won't keep.
!awk -F"_" \
'BEGIN{OFS=";"}{ if ( substr($1,0,1) == ">"){ print $0 ";barcodelabel=" $11 } else { print $0 } }' \
../data/Seq_data/QCSeqs/16S_QC.fasta | \
sed 's/;>//' > ../data/Seq_data/QCSeqs/16S_QC.usearch.fasta

In [14]:
!head ../data/Seq_data/QCSeqs/16S_QC.usearch.fasta

>HWI-M04026_146_000000000-B58PG_1_1101_13055_1749_1_N_0_TGCTCGTAGATCGTGT;barcodelabel=TGCTCGTAGATCGTGT
CGTAGGGGGCAAGCGTTGTCCGGAATCATTGGGCGTAAAGAGCGTGTAGGCGGCCCGATAAGTCCGCTGTGAAAGTCCAGGGCTCAACTCTGGAATGCCGGTGGATACTGTCGGGCTCGAATCCGGAAGAGGTGAGTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCAATGGCGAAGGCAGCTCACTGGGACGGTATTGACGCTGAGACGCGAAAGCGTGGGGAGCAAACA
>HWI-M04026_146_000000000-B58PG_1_1106_17574_21259_1_N_0_CGTAGCGAACGACGTG;barcodelabel=CGTAGCGAACGACGTG
CGTAGGGGGCAAGCGTTGTCCGGAATCATTGGGCGTAAAGAGCGTGTAGGCGGCCCGATAAGTCCGCTGTGAAAGTCCAGGGCTCAACTCTGGAATGCCGGTGGATACTGTCGGGCTCGAATCCGGAAGAGGTGAGTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCAATGGCGAAGGCAGCTCACTGGGACGGTATTGACGCTGAGACGCGAAAGCGTGGGGAGCAAACA
>HWI-M04026_146_000000000-B58PG_1_2104_27344_16447_1_N_0_ACGTGCGCGACACTGA;barcodelabel=ACGTGCGCGACACTGA
CGTAGGGGGCAAGCGTTGTCCGGAATCATTGGGCGTAAAGAGCGTGTAGGCGGCCCGATAAGTCCGCTGTGAAAGTCCAGGGCTCAACTCTGGAATGCCGGTGGATACTGTCGGGCTCGAATCCGGAAGAGGTGAGTGGAATTCCTGGTGTAGCGGTGAAATGCGCAGATATCAGGAGGAACACCAATG

In [15]:
!usearch -derep_fulllength ../data/Seq_data/QCSeqs/16S_QC.usearch.fasta -fastaout ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.fasta -sizeout -threads 4

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

00:05 585Mb  100.0% Reading ../data/Seq_data/QCSeqs/16S_QC.usearch.fasta
00:08 608Mb 1379867 seqs, 1926 uniques, 650 singletons (33.7%)          
00:08 608Mb Min size 1, median 7, max 154896, avg 716.44
00:08 608Mb  100.0% Writing ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.fasta


In [16]:
!head ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.fasta

>HWI-M04026_146_000000000-B58PG_1_1101_25849_8984_1_N_0_TGCTCGTAGATCGTGT;barcodelabel=TGCTCGTAGATCGTGT;size=154896;
CGTAGGGCGCAAGCGTTATCCGGAATTATTGGGCGTAAAGAGCTCGTAGGCGGTTTGTCGCGTCTGCCGTGAAAGTCCGG
GGCTCAACTCCGGATCTGCGGTGGGTACGGGCAGACTAGAGTGATGTAGGGGAGACTGGAATTCCTGGTGTAGCGGTGAA
ATGCGCAGATATCAGGAGGAACACCGATGGCGAAGGCAGGTCTCTGGGCATTAACTGACGCTGAGGAGCGAAAGCATGGG
GAGCGAACA
>HWI-M04026_146_000000000-B58PG_1_1101_8837_4649_1_N_0_TGCTCGTAGATCGTGT;barcodelabel=TGCTCGTAGATCGTGT;size=110072;
CGAAGGGGGCTAGCGTTGCTCGGAATCACTGGGCGTAAAGGGTGCGTAGGCGGGTCTTTAAGTCAGGGGTGAAATCCTGG
AGCTCAACTCCAGAACTGCCTTTGATACTGAGGATCTTGAGTTCGGGAGAGGTGAGTGGAACTGCGAGTGTAGAGGTGAA
ATTCGTAGATATTCGCAAGAACACCAGTGGCGAAGGCGGCTCACTGGCCCGATACTGACGCTGAGGCACGAAAGCGTGGG
GAGCAAACA


In [17]:
# Sequences are sorted by size
# We are excluding the singletons here with minsize =2
!usearch -sortbysize ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.fasta -fastaout ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.sorted.fasta -minsize 2

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

00:00 2.9Mb  100.0% Reading ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.fasta
00:00 2.2Mb Getting sizes                                                      
00:00 2.2Mb Sorting 1276 sequences
00:00 2.2Mb  100.0% Writing output


In [20]:
# Here we are just creating the centroids, or "seeds" for future clustering
# Default is 97% minimum ID - not recommended to use more than 97%.

!usearch -cluster_otus ../data/Seq_data/QCSeqs/16S_QC.usearch.unique.sorted.fasta -otus ../data/Seq_data/QCSeqs/otus.fasta

usearch v8.0.1623_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-15 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

Licensed to: tlw59@cornell.edu

00:01 6.3Mb  100.0% 514 OTUs, 103 chimeras (8.1%)


In [52]:
# This is where the actual OTUs are being assigned. We choose 97% sequence ID threshold here.
# This might take a while - Depending ont he clustering algorithm, like pairwise... 
# it would take, like, days on the same number of processors.
# This is why usearch (centroid-based) is so much better
# But is it more biologically relevant? ... maybe, maybe not.
# Edgar is showing it's not that bad.

# We take our total QC data
# We compare it to the otusn.pick.fasta database we made above
# We produce a readmap.uc file which tells us how the reads from our finalQC file map to the otusn seed database.

!usearch9 -usearch_global ../data/Seq_data/QCSeqs/16S_QC.usearch.fasta -db ../data/Seq_data/QCSeqs/otus.fasta \
-strand plus -id 0.97 -otutabout ../data/Seq_data/QCSeqs/otu_table.txt -biomout ../data/Seq_data/QCSeqs/otu_table.biom -threads 4

usearch v9.2.64_i86osx32, 4.0Gb RAM (17.2Gb total), 4 cores
(C) Copyright 2013-16 Robert C. Edgar, all rights reserved.
http://drive5.com/usearch

License: twhitman@wisc.edu

00:00 1.9Mb   100.0% Reading ../data/Seq_data/QCSeqs/otus.fasta
00:00 1.7Mb   100.0% Masking (fastnucleo)                      
00:00 2.6Mb   100.0% Word stats          
00:00 2.6Mb   100.0% Alloc rows
00:00 3.1Mb   100.0% Build index
00:29 40Mb    100.0% Searching 16S_QC.usearch.fasta, 99.5% matched 
1373016 / 1379867 mapped to OTUs (99.5%)                          
00:29 40Mb   Writing ../data/Seq_data/QCSeqs/otu_table.txt
00:29 40Mb   Writing ../data/Seq_data/QCSeqs/otu_table.txt ...done.
00:29 40Mb   Writing ../data/Seq_data/QCSeqs/otu_table.biom
00:29 40Mb   Writing ../data/Seq_data/QCSeqs/otu_table.biom ...done.


In [58]:
# Issues with biom table formatting
!if [ -f ../data/Seq_data/QCSeqs/otu_table.biom ]; then rm ../data/Seq_data/QCSeqs/otu_table.biom; fi #This is to mitigate a biom bug
!biom convert -i ../data/Seq_data/QCSeqs/otu_table.txt -o ../data/Seq_data/QCSeqs/otu_table.biom --table-type="OTU table" --to-json
!biom summarize-table -i ../data/Seq_data/QCSeqs/otu_table.biom -o ../data/Seq_data/QCSeqs/otu_table.summary

In [59]:
# This tells us the overall data info
# Num obs = OTUs
# total count = total seqs
# Not awful, but might want more depth
# However, remember we ran duplicates of every soil, so we may have ~double the apparent sequences
# Spot checking the lowest ones reveals they are blanks
# This might be pretty good.
# Still, relatively low output - will contrast with dada2 clustering algorithm

!cat ../data/Seq_data/QCSeqs/otu_table.summary

Num samples: 238
Num observations: 514
Total count: 1373016
Table density (fraction of non-zero values): 0.233

Counts/sample summary:
 Min: 1.0
 Max: 56825.0
 Median: 3684.500
 Mean: 5768.975
 Std. dev.: 7189.242
 Sample Metadata Categories: None provided
 Observation Metadata Categories: None provided

Counts/sample detail:
 GTCTATGATCGACGAG: 1.0
 AACGCTGATGCGTACG: 1.0
 CGTAGCGAATCGTACG: 2.0
 AACGCTGACGTTACTA: 2.0
 ACTCACTGGACACTGA: 6.0
 TAGTCTCCGGATATCT: 7.0
 TAGTCTCCCTACTATA: 8.0
 GTCTGCTACTGCGTGT: 34.0
 GATCTACGCGTTACTA: 75.0
 ATAGTACCGGATATCT: 116.0
 GATCTACGTAGCGAGT: 118.0
 GTCTGCTATACGAGAC: 123.0
 GTCTATGATCATCGAG: 211.0
 CGAAGTATACTATCTG: 494.0
 TATAGCGATGCGTACG: 539.0
 ACGCTACTACGTCTCG: 596.0
 ACTACGACCGTCGCTA: 615.0
 GTAACGAGCGTGAGTG: 641.0
 GTCTATGAGACACTGA: 659.0
 GTAACGAGCTAGAGCT: 691.0
 ACGTGCGCACGTCTCG: 738.0
 CTCGACTTTAGTGTAG: 740.0
 ACGTGCGCCGTGAGTG: 798.0
 TATAGCGAGGATATCT: 855.0
 ACGCTACTGCTCTAGT: 907.0
 GCGTATACCTACTATA: 952.0
 CGAGCGACATATACAC: 954.0
 GTCTATGAGCTC

In [ ]:
# We could think about prepping a MiSeq nano run with a library of the lowest abundance samples,
# with a few of the high-abundance samples thrown in to check for consistenct btw runs